# Analysis for each session

Our experiment consists in testing a decision-making task and self-reports with volunteer participants in order to study how they influence each other. The experiment runs online through Jatos platform and last 20 sessions: 2 sessions per day: one in the morning and the other in the after, summing up to **10 consecutive days**. We provide to each participant a personal link that she is going to use once per session. Each session of the game starts with **3/4 questions about their mood, the quality of their sleep, the enjoyment of the food, and their stress**. They answer to the self-reports positioning a button in a continuous scale with sad and smile emoji faces at the ends. After the questionnaire, we present them noisy stimuli, and the participant has to answer deciding if the stimulus was towards the left or towards the right. If the participant chooses the correct answer, then she receives 3 points. A bad choice does not add points.

The task is the **dots task** where the participant sees two circles with white dots in a black background, and she has to decide which of the circles has more dots as in Fig. 1. One of the circles always contains 50 and the other a greater amount of dots making impossible to count them to answer. The normalized difference between is computed as the **signal** present in the trial. We sort at random which circle will contain more dots in a way to present a balanced quantity of left and rightward stimuli per stage. 

In [ ]:
import pandas as pd
import os
import json
import numpy as np
from itertools import groupby
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
import random
import re
import csv
import warnings
warnings.filterwarnings('ignore')
from IPython.display import HTML, display, Image
import statsmodels.formula.api as smf 
import statsmodels.api as sm

mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.linewidth'] = 3
#mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 4
#mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.width'] = 4
mpl.rcParams['ytick.minor.width'] = 2

fday = 1
fsession = 1

In [ ]:
current_path = os.path.abspath(os.getcwd())
parent_path = os.path.abspath(os.path.join(current_path, os.pardir))
grand_parent_path = os.path.abspath(os.path.join(parent_path, os.pardir))
main_path = os.path.abspath(os.path.join(grand_parent_path, os.pardir))

path_results = main_path+'/results/dots/'
path_plots = main_path+'/scripts/analysis/figures/'

In [ ]:
main_path

In [ ]:
import sys
sys.path.insert(1, main_path+'/src')
import my_functions as myf

In [ ]:
path = main_path+'/data/jatos_dots_data/tanda1/day'+str(fday)+'/session'+str(fsession)+'/'
filename_average= path_results+'across_sessions/average_Day'+str(fday)+'_Sess'+str(fsession)+'.json'

In [ ]:
Image(filename= main_path+'/docs/example.png') 

**Figure 1**: Example stimulus that we display in the experiment.

In [ ]:
# list of participants 
participants = [3051+i for i in range(28)]

In [ ]:
data_files = [f for f in os.listdir(path) if f.endswith('_day'+str(fday)+'_session'+str(fsession))]

In [ ]:
# sort files
subj_data = [int(re.search('%s(.*)%s' % ('', '_day'), f).group(1)) for f in data_files]
sorted_subj_data = sorted(subj_data)
index_subj_data = [subj_data.index(elem) for elem in sorted_subj_data]
sorted_data_files = [data_files[i] for i in index_subj_data]

In [ ]:
# reports, practice, deterministic and stochastic df for each participant

Rdf,Pdf,Ddf,Sdf = {},{},{},{}
for name in sorted_subj_data: 
    Rdf[name] = {}
    Pdf[name] = pd.DataFrame()
    Ddf[name] = pd.DataFrame()
    Sdf[name] = pd.DataFrame()

In [ ]:
rep_list = []
ind = -1
for ses in sorted_data_files:
    ind += 1
    data = [] 
    for line in open(path+ses, 'r'):
        if line.strip():
            data.append(json.loads(line))

    if fday==1 and fsession==1:
        if len(data)==5:
            demographics = data[0]
            reports = data[1]
            practice = data[2]
            deterministic = data[3]
            stochastic = data[4]
        else:
            reports = data[1]
            practice = data[len(data)-3]
            deterministic = data[len(data)-2]
            stochastic = data[len(data)-1]
            print('this participant ',part,' has repeated some stage')
    else:
        if len(data)==4:
            reports = data[0]
            practice = data[1]
            deterministic = data[2]
            stochastic = data[3]
        elif len(data)>4:
            reports = data[0]
            practice = data[len(data)-3]
            deterministic = data[len(data)-2]
            stochastic = data[len(data)-1]
            print('participant '+ str(sorted_subj_data[ind])+' has repeated practice '+str(len(data)-3))
            pract = []
            for j in range(len(data)):
                if len(data[j])==10:
                    pract.append(data[j])
        else: 
            for dd in range(len(data)):
                print(len(data[dd]))
            reports = data[0]
            practice = data[1]
            deterministic = data[2]
            print('participant '+ str(sorted_subj_data[ind])+' missed stochastic')        
    part = reports['userID']
    if reports['sessionID']%2==0:
        session = 2
    else:
        session = 1
    
    if (part!=str(sorted_subj_data[ind])):
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('INCORRECT')
        print('participant',part,'file participant',sorted_subj_data[ind])
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
    if (session!=fsession):
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('INCORRECT')
        print(part, 'session',session,'file session',fsession)
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')
        print('!!!!!!!!!!!!!!!!!!')

    Rdf[sorted_subj_data[ind]] = reports
    Pdf[sorted_subj_data[ind]] = pd.DataFrame.from_dict(practice)
    Ddf[sorted_subj_data[ind]] = pd.DataFrame.from_dict(deterministic)
    Sdf[sorted_subj_data[ind]] = pd.DataFrame.from_dict(stochastic)
    rep_list.append(reports)

## Reports

The first part of the experiment consists in answering 3 or 4 self-reports by changing a buttom in a continuos slider between a sad and a smile emoji faces.

**Morning session questions**:

- Mood report: ¿Cómo te has sentido esta tarde? (How did you feel this morning?) 
- Sleep report: ¿Cómo dormiste anoche? (How did you sleep the past night?)
- Food report: ¿Cómo disfrutaste la última comida/snack? (How did you enjoy the last meal/snack?)
- Stress report: ¿Cómo has sentido la carga de problemas personales y laborales esta mañana? (How did you feel the personal and working problems burden this morning?)

**Afternoon session questions**:

- Mood report: ¿Cómo te has sentido esta tarde? (How did you feel this afternoon?) 
- Food report: ¿Cómo disfrutaste la última comida/snack? (How did you enjoy the last meal/snack?)
- Stress report: ¿Cómo has sentido la carga de problemas personales y laborales esta tarde? (How did you feel the personal and working problems burden this afternoon?)

Since the stress report ranks high (low) when participants are less (more) stressed, we compute the *real_stress=1-stress*.

In [ ]:
for part in sorted_subj_data:
    mood = Rdf[part]['mood']
    stress = Rdf[part]['stress']
    real_stress = 1-stress
    food = Rdf[part]['food']
    if fsession==1:
        sleep = Rdf[part]['sleep']
        if mood<0.15 and real_stress<0.15 and food<0.15 and sleep<0.15:
            print('Here we printed the "abnormal" self-report answers to e-mail the participant in order to get \
            knowledge of the reason.')
            print('-',part, 'has all reports below 0.15: ',Rdf[part])
        elif mood>0.85 and real_stress>0.85 and food>0.85 and sleep>0.85:
            print('Here we printed the "abnormal" self-report answers to e-mail the participant in order to get \
            knowledge of the reason.')
            print('-',part, 'has all reports over 0.85: ',Rdf[part])
    else: 
        if mood<0.15 and real_stress<0.15 and food<0.15:
            print('Here we printed the "abnormal" self-report answers to e-mail the participant in order to get \
            knowledge of the reason.')
            print('-',part, 'has all reports below 0.15: ',Rdf[part])
        elif mood>0.85 and real_stress>0.85 and food>0.85:
            print('Here we printed the "abnormal" self-report answers to e-mail the participant in order to get \
            knowledge of the reason.')
            print('-',part, 'has all reports over 0.85: ',Rdf[part])

In [ ]:
Reportsdf = pd.DataFrame(rep_list)

In [ ]:
Reportsdf['real_stress'] = 1-Reportsdf['stress']

In [ ]:
Reportsdf.head(27)

**Table 1**: Self-reports answered by each participant in the current session. Column names: 
- mood, sleep, food, stress: raw self-reports
- sessionID: session number = 2*Day-2+Session, where Day takes values {1,2,...,10} and Session takes values {1,2}
- userID: participant number ID
- date: browser's time zone (Spain in all cases)
- real_stress: 1-stress

In [ ]:
# path to save results
path_fit = path_results+'day'+str(fday)+'/session'+str(fsession)+'/'

## Practice stage

After answering the self-report's questionnaire, it starts the dots task composed by three stages: practice, deterministic and stochastic. The practice stage is present in all sessions and consists of 10 trials with increasing (decreasing) difficulty if the participant answered correctly (incorrectly) in the previous trial. 

In [ ]:
ind = -1
fig, ax = plt.subplots(7,4,figsize=(18,25))
plt.subplots_adjust(wspace = 0.4)
plt.subplots_adjust(hspace = 0.3)
for part in sorted_subj_data:
    Pdf[part]['signal'] = np.abs(Pdf[part]['dots_num_left']-Pdf[part]['dots_num_right'])
    Presult = list(Pdf[part].discrimination_is_correct)
    Psignal = list(Pdf[part].signal)

    ind += 1
    ind1 = ind%7
    ind2 = int(round(ind/7,1))
    ax[ind1,ind2].set_title('participant:'+str(part))
    ax[ind1,ind2].plot(np.arange(1,11),np.array(Psignal)/100,'r')
    ax[ind1,ind2].set_ylim(0.01,0.27)
    ax[ind1,0].set_ylabel('Signal')
    ax[6,ind2].set_xlabel('Practice trial number')
    ax[6,3].axis('off')
plt.show()

**Figure 2**: Practice signal vs trial number for each participant. It always starts with 0.2 signal (20 dots of difference between the two circles). Minumum possible signal: 0.02 (2 dots of difference).

## Deterministic and stochastic stages

In some trials, the participant has a third option to choose: **the opt-out option**. By choosing it, the subject skips the decision and passes to the next trial. Each session of the game has two stages, each of them with 120 trials. The difference between the two stages is the action of the opt-out option. In the first stage, the opt-out option returns a fix number of 2 points. We called this stage: **Deterministic Optout (DO) stage**. While, in the second stage, the 80% of the trials the opt-out option returns 3 points, while the other 20% does not return points. The name of this stage is: **Stochastic Optout (SO) stage** and we use it to study risk aversion.

Before each stimulus of the DO stage, we display either a question mark or a cross, in order to inform to the participant if she will or will not have the opt-out option. For the SO stage, we change the question mark with the image of a dice, indicating the stochastic character of the trial.

In order to study the psychometric variables, we slip the trials into **4 difficulties** (15 trials each) with signals: 0.02, 0.06, 0.1, 0.14. That is, 2, 6, 10 and 14 dots of difference between the circles. The trial difficulty presentation is also sorted randomly. 

In [ ]:
# deterministic dataframe
for part in sorted_subj_data:
    # discrimination RT
    discrimination_RT = np.array(list(Ddf[part].discrimination_t_keydown))-np.array(list(Ddf[part].t_offset))
    Ddf[part]["discrimination_RT"] = discrimination_RT 
    Ddf[part]['signal'] = np.abs(Ddf[part]['dots_num_left']-Ddf[part]['dots_num_right'])
    Dsignal = np.array(np.abs(Ddf[part]['dots_num_left']-Ddf[part]['dots_num_right']))/100
    Dorientation = Ddf[part]['orientation']
    signedDsignal = []
    ind = -1
    for elem in Dorientation:
        ind += 1
        if elem:
            signedDsignal.append(Dsignal[ind])
        else: 
            signedDsignal.append(-Dsignal[ind])  
    # signed signal (neg signal to the left and pos signal tothe right)
    Ddf[part]['signed_signal'] = signedDsignal
    Dcorrect_list = list(Ddf[part]['discrimination_is_correct'])
    Doptout_list = list(Ddf[part]["optout"])
    Dbool_correct_list = [elem==1 for elem in Dcorrect_list]
    Dbool_orientation_list = [elem==1 for elem in Dorientation]
    Dxor = np.logical_xor(Dbool_correct_list,np.logical_not(Dbool_orientation_list))
    Dresp = []
    ind = -1
    for elem in Doptout_list:
        ind += 1
        if elem==False and Dxor[ind]==True:
            Dresp.append(1)
        elif elem==False and Dxor[ind]==False:
            Dresp.append(0)
        elif elem==True: 
            Dresp.append(2)
    # 1 for rightward answers, 0 for left
    Ddf[part]['resp_is_R'] = [elem==1 for elem in Dresp]
    Ddf[part]['resp_is_R'] = Ddf[part]['resp_is_R'].astype(int)
    # 1 for rightward answers, 0 for left, and 2 for optout
    Ddf[part]['answer'] = Dresp

In [ ]:
Dtoffset = []
for part in sorted_subj_data:
    Dtoffset.append(Ddf[part]['t_offset'])
    
fig = plt.figure(1, figsize=(9, 6))
ax = fig.add_subplot(111)
bp = ax.boxplot(Dtoffset)
ax.set_xticks(np.arange(1,28))
ax.set_xlabel('participants')
ax.set_ylabel('deterministic time offset (stim display)')
ax.set_xticklabels(sorted_subj_data, rotation = 90)

plt.show()

In [ ]:
Ddiscrimination_t_keydown = []
for part in sorted_subj_data:
    Ddiscrimination_t_keydown.append(Ddf[part]['discrimination_t_keydown'])
    
fig = plt.figure(1, figsize=(9, 6))
ax = fig.add_subplot(111)
bp = ax.boxplot(Ddiscrimination_t_keydown)
ax.set_xticks(np.arange(1,28))
ax.set_xlabel('participants')
ax.set_ylabel('reaction time (discrimination_t_keydown)')
ax.set_xticklabels(sorted_subj_data, rotation = 90)

plt.show()

In [ ]:
dead_time = np.array(Ddf[3051].discrimination_t_onset)-np.array(Ddf[3051].t_offset)

In [ ]:
delay_time = np.array(Ddf[3051].t_offset)-300

In [ ]:
plt.scatter(delay_time,dead_time)
plt.title('participant 3051')
plt.xlabel('delay time (ms)')
plt.ylabel('dead time (ms)')
plt.show()

In [ ]:
# discard t offset greater than 350 ms
for name in sorted_subj_data: 
    Ddf[name] = Ddf[name][Ddf[name]['t_offset']<=350]

In [ ]:
# deterministic non-optout & optout df
Ddf_no,Ddf_oo = {},{}
for name in sorted_subj_data: 
    Ddf_no[name] = pd.DataFrame()
    Ddf_oo[name] = pd.DataFrame()
for part in sorted_subj_data:
    Ddf_no[part] = Ddf[part][(Ddf[part]['focus']==0)]
    Ddf_oo[part] = Ddf[part][(Ddf[part]['focus']==1)]

In [ ]:
# stochastic dataframe
for part in sorted_subj_data:
    # discrimination RT
    Sdiscrimination_RT = np.array(list(Sdf[part].discrimination_t_keydown))-np.array(list(Sdf[part].t_offset))
    Sdf[part]["discrimination_RT"] = Sdiscrimination_RT 
    Sdf[part]['signal'] = np.abs(Sdf[part]['dots_num_left']-Sdf[part]['dots_num_right'])
    Ssignal = np.array(np.abs(Sdf[part]['dots_num_left']-Sdf[part]['dots_num_right']))/100
    Sorientation = Sdf[part]['orientation']
    signedSsignal = []
    ind = -1
    for elem in Sorientation:
        ind += 1
        if elem:
            signedSsignal.append(Ssignal[ind])
        else: 
            signedSsignal.append(-Ssignal[ind])  
    # signed signal (neg signal to the left and pos signal tothe right)
    Sdf[part]['signed_signal'] = signedSsignal
    Scorrect_list = list(Sdf[part]['discrimination_is_correct'])
    Soptout_list = list(Sdf[part]["optout"])
    Sbool_correct_list = [elem==1 for elem in Scorrect_list]
    Sbool_orientation_list = [elem==1 for elem in Sorientation]
    Sxor = np.logical_xor(Sbool_correct_list,np.logical_not(Sbool_orientation_list))
    Sresp = []
    ind = -1
    for elem in Soptout_list:
        ind += 1
        if elem==False and Sxor[ind]==True:
            Sresp.append(1)
        elif elem==False and Sxor[ind]==False:
            Sresp.append(0)
        elif elem==True: 
            Sresp.append(2)
    # 1 for rightward answers, 0 for left
    Sdf[part]['resp_is_R'] = [elem==1 for elem in Sresp]
    Sdf[part]['resp_is_R'] = Sdf[part]['resp_is_R'].astype(int)
    # 1 for rightward answers, 0 for left, and 2 for optout
    Sdf[part]['answer'] = Sresp

In [ ]:
Stoffset = []
for part in sorted_subj_data:
    Stoffset.append(Sdf[part]['t_offset'])
    
fig = plt.figure(1, figsize=(9, 6))
ax = fig.add_subplot(111)
bp = ax.boxplot(Stoffset)
ax.set_xticks(np.arange(1,28))
ax.set_xlabel('participants')
ax.set_ylabel('stochastic time offset (stim display)')
ax.set_xticklabels(sorted_subj_data, rotation = 90)

plt.show()

In [ ]:
# discard t offset greater than 350 ms
for name in sorted_subj_data: 
    Sdf[name] = Sdf[name][Sdf[name]['t_offset']<=350]

In [ ]:
# stochastic non-optout & optout df
Sdf_no,Sdf_oo = {},{}
for name in sorted_subj_data: 
    Sdf_no[name] = pd.DataFrame()
    Sdf_oo[name] = pd.DataFrame()
for part in sorted_subj_data:
    Sdf_no[part] = Sdf[part][(Sdf[part]['focus']==0)]
    Sdf_oo[part] = Sdf[part][(Sdf[part]['focus']==1)]

In [ ]:
# non-optout df for each participant
df_no = {}
for name in sorted_subj_data: 
    df_no[name] = pd.DataFrame()
for part in sorted_subj_data:
    df_no[part] = pd.concat([Ddf_no[part],Sdf_no[part]])

In [ ]:
# all data together df for each participant
df_all = {}
for name in sorted_subj_data: 
    df_all[name] = pd.DataFrame()
for part in sorted_subj_data:
    df_all[part] = pd.concat([Ddf[part],Sdf[part]])

In [ ]:
## DO NOT RUN AGAIN

## toffset 

median_toffset,max_toffset = [],[]
for part in sorted_subj_data:
    toffset = df_all[part]['t_offset']
    median_toffset.append(np.median(toffset))
    max_toffset.append(np.max(toffset))
    
# write the result in file
dict_ = {
    "median_toffset":median_toffset,
    "max_toffset":max_toffset
}
# Serializing json  
json_object = json.dumps(dict_) 
    
# append to the dictionary in the existing file
with open(filename_average) as outfile:
    old_data = json.load(outfile)
old_data.update(dict_)
with open(filename_average, 'w') as outfile:
    json.dump(old_data, outfile)

### Performance vs. difficulty

In [ ]:
ind = -1
fig, ax = plt.subplots(7,4,figsize=(18,25))
plt.subplots_adjust(wspace = 0.3)
plt.subplots_adjust(hspace = 0.3)
middle_perf_no = []
subj_perf_diff_oo,Ssubj_perf_diff_oo,subj_perf_diff_no = [[] for _ in range(3)]
subj_perf_oo,Ssubj_perf_oo,subj_perf_no = [[] for _ in range(3)]
for part in sorted_subj_data:
    perf_diff_oo,perf_diff_no,Sperf_diff_oo,se_perf_diff_oo,Sse_perf_diff_oo,\
    se_perf_diff_no,lnum_trials_oo,lSnum_trials_oo,lnum_trials_no=[[] for _ in range(9)]
    for diff in range(4):
        # correct deterministic
        subset_oo = Ddf_oo[part][(Ddf_oo[part]["difficulty"]==diff) & (Ddf_oo[part]["optout"]==0)]  
        result_oo = np.array(subset_oo.discrimination_is_correct)
        num_trials_oo = len(result_oo)
        lnum_trials_oo.append(num_trials_oo)
        if num_trials_oo:
            perf_diff_oo.append(100*np.sum(result_oo)/num_trials_oo)
            se_perf_diff_oo.append(100*np.sqrt(perf_diff_oo[diff]/100 *(1-perf_diff_oo[diff]/100)/num_trials_oo))
        else:
            perf_diff_oo.append(np.nan)
            se_perf_diff_oo.append(np.nan)
        # correct stochastic
        Ssubset_oo = Sdf_oo[part][(Sdf_oo[part]["difficulty"]==diff) & (Sdf_oo[part]["optout"]==0)]  
        Sresult_oo = np.array(Ssubset_oo.discrimination_is_correct)
        Snum_trials_oo = len(Sresult_oo)
        lSnum_trials_oo.append(Snum_trials_oo)
        if Snum_trials_oo:
            Sperf_diff_oo.append(100*np.sum(Sresult_oo)/Snum_trials_oo)
            Sse_perf_diff_oo.append(100*np.sqrt(Sperf_diff_oo[diff]/100 *(1-Sperf_diff_oo[diff]/100)/Snum_trials_oo))
        else: 
            Sperf_diff_oo.append(np.nan)
            Sse_perf_diff_oo.append(np.nan)
        # non optout
        subset_no = df_no[part][(df_no[part]["difficulty"]==diff)]
        result_no = np.array(subset_no.discrimination_is_correct)
        num_trials_no = len(result_no)
        lnum_trials_no.append(num_trials_no)
        perf_diff_no.append(100*np.sum(result_no)/num_trials_no)
        se_perf_diff_no.append(100*np.sqrt(perf_diff_no[diff]/100 *(1-perf_diff_no[diff]/100)/num_trials_no))
        if diff==1:
            middle_perf_no.append(perf_diff_no[diff])
            
    subj_perf_diff_oo.append(perf_diff_oo)
    Ssubj_perf_diff_oo.append(Sperf_diff_oo)
    subj_perf_diff_no.append(perf_diff_no)
    
    subj_perf_oo.append(np.nanmean(perf_diff_oo))
    Ssubj_perf_oo.append(np.nanmean(Sperf_diff_oo))
    subj_perf_no.append(np.nanmean(perf_diff_no))
        
    # write the result in file
    filename=path_fit+'diff_Sub'+str(part)+'_Day'+str(fday)+'_Sess'+str(fsession)+'.json'
    dict_ = {
        "Dperf_oo":perf_diff_oo,
        "Dse_perf_oo":se_perf_diff_oo,
        "Sperf_oo":Sperf_diff_oo,
        "Sse_perf_oo":Sse_perf_diff_oo,
        "perf_no":perf_diff_no,
        "se_perf_no":se_perf_diff_no,  
        "Dn_trials_oo":lnum_trials_oo,
        "Sn_trials_oo":lSnum_trials_oo,
        "NOn_trials":lnum_trials_no
    }
    # Serializing json  
    json_object = json.dumps(dict_) 

    # Writing to sample.json 
    with open(filename, "w") as outfile: 
        outfile.write(json_object) 

    ind += 1
    ind1 = ind%7
    ind2 = int(round(ind/7,1))
    ax[ind1,ind2].set_ylim(0,105)
    ax[ind1,ind2].set_title('participant:'+str(part))
    ax[ind1,ind2].errorbar(np.arange(1,5),perf_diff_oo,yerr=se_perf_diff_oo,color='r',ls='-')
    ax[ind1,ind2].errorbar(np.arange(1,5),perf_diff_no,yerr=se_perf_diff_no,color='g',ls='-')
    ax[ind1,ind2].errorbar(np.arange(1,5),Sperf_diff_oo,yerr=Sse_perf_diff_oo,color='b',ls='-')
    ax[ind1,0].set_ylabel('Performance')
    ax[6,ind2].set_xlabel('Difficulty')
    ax[ind1,ind2].set_xticks(np.arange(1,5))
    ax[0,0].legend(("DO optout","non-optout","SO optout"),loc='best', shadow=True)
    ax[6,3].axis('off')
    
# write the result in file
dict_ = {
    "participantID":sorted_subj_data,
    "sessionID":[reports['sessionID']]*(len(sorted_subj_data)),
    "Dsubj_perf_oo":subj_perf_oo,
    "Ssubj_perf_oo":Ssubj_perf_oo,
    "subj_perf_no":subj_perf_no
}
# Serializing json  
json_object = json.dumps(dict_) 

# Writing to sample.json 
with open(filename_average, "w") as outfile: 
    outfile.write(json_object) 
plt.show()

**Figure 3**: Mean and standard deviation performance vs difficulty for each participant. Green: non-optout trials. Red: DO optout trials. Blue: SO optout trials.

In [ ]:
mean_subj_perf_oo,Smean_subj_perf_oo,mean_subj_perf_no,se_subj_perf_oo,Sse_subj_perf_oo,se_subj_perf_no = [[] for _ in range(6)]
for diff in range(4):
    mean_subj_perf_oo.append(np.nanmean(np.array(subj_perf_diff_oo)[:,diff]))
    Smean_subj_perf_oo.append(np.nanmean(np.array(Ssubj_perf_diff_oo)[:,diff]))
    mean_subj_perf_no.append(np.nanmean(np.array(subj_perf_diff_no)[:,diff]))
    se_subj_perf_oo.append(np.nanstd(np.array(subj_perf_diff_oo)[:,diff])/np.sqrt(len(sorted_subj_data)))
    Sse_subj_perf_oo.append(np.nanstd(np.array(Ssubj_perf_diff_oo)[:,diff])/np.sqrt(len(sorted_subj_data)))
    se_subj_perf_no.append(np.nanstd(np.array(subj_perf_diff_no)[:,diff])/np.sqrt(len(sorted_subj_data)))
plt.errorbar(np.arange(1,5),mean_subj_perf_oo,se_subj_perf_oo,c='r')
plt.errorbar(np.arange(1,5),mean_subj_perf_no,se_subj_perf_no,c='g')
plt.errorbar(np.arange(1,5),Smean_subj_perf_oo,Sse_subj_perf_oo,c='b')
plt.legend(("DO optout","non-optout","SO optout"),loc='best', shadow=True)
plt.xlabel('Difficulty')
plt.ylabel('Population mean perf')
plt.show()

**Figure 4**: Mean and standard error performance vs difficulty across participants. Green: non-optout trials. Red: DO optout trials. Blue: SO optout trials.

### Response time (RT) vs. difficulty

As a cue of self-confidence, we study the median reaction time in correct trials for each of the three type of trials. 

In [ ]:
ind = -1
fig, ax = plt.subplots(7,4,figsize=(18,25))
plt.subplots_adjust(wspace = 0.3)
plt.subplots_adjust(hspace = 0.3)
subj_RT_oo,Ssubj_RT_oo,subj_RT_no = [[] for _ in range(3)]
OKsubj_RT_oo,OKSsubj_RT_oo,OKsubj_RT_no = [[] for _ in range(3)]
NOKsubj_RT_oo,NOKSsubj_RT_oo,NOKsubj_RT_no = [[] for _ in range(3)]
subj_RT_diff_oo,Ssubj_RT_diff_oo,subj_RT_diff_no,subj_medianRT = [[] for _ in range(4)]
for part in sorted_subj_data:
    # list with RT in every trials for this participant
    RT_list = list(df_no[part]['discrimination_t_keydown'])+list(Ddf_oo[part]['discrimination_t_keydown'])+\
    list(Sdf_oo[part]['discrimination_t_keydown'])
    # global median RT for this participant
    medianRT = np.nanmedian(RT_list)
    # list with the global median RT for every participant
    subj_medianRT.append(medianRT)
    
    ## non-optout trial
    subj_RT_no.append(np.nanmedian(list(df_no[part]['discrimination_t_keydown']))/medianRT)
    ## optout trials
    subset_do = Ddf_oo[part][Ddf_oo[part]["optout"]==1]
    subset_so = Sdf_oo[part][Sdf_oo[part]["optout"]==1]
    # lists with the normalized median RT for optout trials
    subj_RT_oo.append(np.nanmedian(list(subset_do['discrimination_t_keydown']))/medianRT)
    Ssubj_RT_oo.append(np.nanmedian(list(subset_so['discrimination_t_keydown']))/medianRT)
    ## correct trials
    OKsubset_no = df_no[part][(df_no[part]["discrimination_is_correct"]==1)]
    OKsubset_do = Ddf_oo[part][(Ddf_oo[part]["discrimination_is_correct"]==1) & (Ddf_oo[part]["optout"]==0)]
    OKsubset_so = Sdf_oo[part][(Sdf_oo[part]["discrimination_is_correct"]==1) & (Sdf_oo[part]["optout"]==0)]
    # lists with the normalized median RT for correct trials
    OKsubj_RT_no.append(np.nanmedian(list(OKsubset_no['discrimination_t_keydown']))/medianRT)
    OKsubj_RT_oo.append(np.nanmedian(list(OKsubset_do['discrimination_t_keydown']))/medianRT)
    OKSsubj_RT_oo.append(np.nanmedian(list(OKsubset_so['discrimination_t_keydown']))/medianRT)
    ## incorrect trials
    NOKsubset_no = df_no[part][(df_no[part]["discrimination_is_correct"]==0)]
    NOKsubset_do = Ddf_oo[part][(Ddf_oo[part]["discrimination_is_correct"]==0) & (Ddf_oo[part]["optout"]==0)]
    NOKsubset_so = Sdf_oo[part][(Sdf_oo[part]["discrimination_is_correct"]==0) & (Sdf_oo[part]["optout"]==0)]
    # lists with the normalized median RT for incorrect trials
    NOKsubj_RT_no.append(np.nanmedian(list(NOKsubset_no['discrimination_t_keydown']))/medianRT)
    NOKsubj_RT_oo.append(np.nanmedian(list(NOKsubset_do['discrimination_t_keydown']))/medianRT)
    NOKSsubj_RT_oo.append(np.nanmedian(list(NOKsubset_so['discrimination_t_keydown']))/medianRT)
    
    RT_diff_oo,SRT_diff_oo,RT_diff_no,RT_diff_noNOK,sd_RT_diff_oo,Ssd_RT_diff_oo,sd_RT_diff_no,sd_RT_diff_noNOK, \
    RT_diff_OKoo,sd_RT_diff_OKoo,SRT_diff_OKoo,Ssd_RT_diff_OKoo = [[] for _ in range(12)]
    for diff in range(4):
        # optout deterministic
        subset_oo = subset_do[(subset_do["difficulty"]==diff)] 
        result_oo = list(subset_oo.discrimination_is_correct)
        num_trials_oo = len(result_oo)
        RT_oo = list(subset_oo.discrimination_t_keydown)
        if num_trials_oo:
            RT_diff_oo.append(np.median(RT_oo)/medianRT)
            sd_RT_diff_oo.append(myf.std_median(RT_oo)/medianRT)
        else:
            RT_diff_oo.append(np.nan)
            sd_RT_diff_oo.append(np.nan)
        # optout stochastic
        Ssubset_oo = subset_so[(subset_so["difficulty"]==diff)]   
        Sresult_oo = list(Ssubset_oo.discrimination_is_correct)
        Snum_trials_oo = len(Sresult_oo)
        SRT_oo = list(Ssubset_oo.discrimination_t_keydown)
        if Snum_trials_oo:
            SRT_diff_oo.append(np.median(SRT_oo)/medianRT)
            Ssd_RT_diff_oo.append(myf.std_median(SRT_oo)/medianRT)
        else: 
            SRT_diff_oo.append(np.nan)
            Ssd_RT_diff_oo.append(np.nan)
        # correct deterministic
        subset_OKoo = OKsubset_do[(OKsubset_do["difficulty"]==diff)] 
        result_OKoo = list(subset_OKoo.discrimination_is_correct)
        num_trials_OKoo = len(result_OKoo)
        RT_OKoo = list(subset_OKoo.discrimination_t_keydown)
        if num_trials_OKoo:
            RT_diff_OKoo.append(np.median(RT_OKoo)/medianRT)
            sd_RT_diff_OKoo.append(myf.std_median(RT_OKoo)/medianRT)
        else:
            RT_diff_OKoo.append(np.nan)
            sd_RT_diff_OKoo.append(np.nan)
        # correct stochastic
        Ssubset_OKoo = OKsubset_so[(OKsubset_so["difficulty"]==diff)]   
        Sresult_OKoo = list(Ssubset_OKoo.discrimination_is_correct)
        Snum_trials_OKoo = len(Sresult_OKoo)
        SRT_OKoo = list(Ssubset_OKoo.discrimination_t_keydown)
        if Snum_trials_OKoo:
            SRT_diff_OKoo.append(np.median(SRT_OKoo)/medianRT)
            Ssd_RT_diff_OKoo.append(myf.std_median(SRT_OKoo)/medianRT)
        else: 
            SRT_diff_OKoo.append(np.nan)
            Ssd_RT_diff_OKoo.append(np.nan)
        # non optout correct
        subset_no = OKsubset_no[(OKsubset_no["difficulty"]==diff)]
        result_no = list(subset_no.discrimination_is_correct)
        num_trials_no = len(result_no)
        RT_no = list(subset_no.discrimination_t_keydown)
        RT_diff_no.append(np.median(RT_no)/medianRT)
        sd_RT_diff_no.append(myf.std_median(RT_no)/medianRT)
        # non optout incorrect
        subset_noNOK = NOKsubset_no[(NOKsubset_no["difficulty"]==diff)]
        #result_noNOK = list(subset_noNOK.discrimination_is_correct)
        #num_trials_noNOK = len(result_noNOK)
        RT_noNOK = list(subset_noNOK.discrimination_t_keydown)
        RT_diff_noNOK.append(np.median(RT_noNOK)/medianRT)
        sd_RT_diff_noNOK.append(myf.std_median(RT_noNOK)/medianRT)
        
    subj_RT_diff_oo.append(RT_diff_OKoo)
    Ssubj_RT_diff_oo.append(SRT_diff_OKoo)
    subj_RT_diff_no.append(RT_diff_no)
      
    # write the result in file
    filename=path_fit+'diff_Sub'+str(part)+'_Day'+str(fday)+'_Sess'+str(fsession)+'.json'
    dict_ = {
        "DRT_oo": RT_diff_oo,
        "DsdRT_oo" : sd_RT_diff_oo,
        "SRT_oo": SRT_diff_oo,
        "SsdRT_oo" : Ssd_RT_diff_oo,
        "DRT_OKoo": RT_diff_OKoo,
        "DsdRT_OKoo" : sd_RT_diff_OKoo,
        "SRT_OKoo": SRT_diff_OKoo,
        "SsdRT_OKoo" : Ssd_RT_diff_OKoo,
        "RT_no": RT_diff_no,
        "sdRT_no" : sd_RT_diff_no, 
        "RT_noNOK": RT_diff_noNOK,
        "sdRT_noNOK" : sd_RT_diff_noNOK 
    }
    
    # append to the dictionary in the existing file
    with open(filename) as outfile:
        old_data = json.load(outfile)
    old_data.update(dict_)
    with open(filename, 'w') as outfile:
        json.dump(old_data, outfile)

    ind += 1
    ind1 = ind%7
    ind2 = int(round(ind/7,1))
    ax[ind1,ind2].set_title('participant:'+str(part))
    ax[ind1,ind2].errorbar(np.arange(1,5),RT_diff_OKoo,yerr=sd_RT_diff_OKoo,c='r',ls='-')
    ax[ind1,ind2].errorbar(np.arange(1,5),RT_diff_no,yerr=sd_RT_diff_no,c='g',ls='-')
    ax[ind1,ind2].errorbar(np.arange(1,5),RT_diff_noNOK,yerr=sd_RT_diff_noNOK,c='m',ls='-')
    ax[ind1,ind2].errorbar(np.arange(1,5),SRT_diff_OKoo,yerr=Ssd_RT_diff_OKoo,c='b',ls='-')
    ax[ind1,0].set_ylabel('median RT')
    ax[6,ind2].set_xlabel('Difficulty')
    ax[ind1,ind2].set_xticks(np.arange(1,5))
    ax[0,0].legend(("DO optout","correct non-optout","incorrect non-optout","SO optout"),loc='best', shadow=True)
    ax[ind1,ind2].set_ylim(0,2)
    ax[6,3].axis('off')
    
# write the result in file
dict_ = {
    "subj_RT_no":subj_RT_no,
    "Dsubj_RT_oo":subj_RT_oo,
    "Ssubj_RT_oo":Ssubj_RT_oo,
    "OKubj_RT_no":OKsubj_RT_no,
    "OKDsubj_RT_oo":OKsubj_RT_oo,
    "OKSsubj_RT_oo":OKSsubj_RT_oo,
    "NOKubj_RT_no":NOKsubj_RT_no,
    "NOKDsubj_RT_oo":NOKsubj_RT_oo,
    "NOKSsubj_RT_oo":NOKSsubj_RT_oo,
    "medianRT":subj_medianRT
}

# append to the dictionary in the existing file
with open(filename_average) as outfile:
    old_data = json.load(outfile)
old_data.update(dict_)
with open(filename_average, 'w') as outfile:
    json.dump(old_data, outfile)
plt.show()

**Figure 5**: Median and standard deviation normalized reaction time vs difficulty for each participant in correct trials where she answer correctly. Green: non-optout correct trials. Magenta: non-optout incorrect trials. Red: DO optout correct trials. Blue: SO optout correct trials.

In [ ]:
mean_subj_RT_oo,Smean_subj_RT_oo,mean_subj_RT_no,se_subj_RT_oo,Sse_subj_RT_oo,se_subj_RT_no = [[] for _ in range(6)]
for diff in range(4):
    mean_subj_RT_oo.append(np.nanmean(np.array(subj_RT_diff_oo)[:,diff]))
    Smean_subj_RT_oo.append(np.nanmean(np.array(Ssubj_RT_diff_oo)[:,diff]))
    mean_subj_RT_no.append(np.nanmean(np.array(subj_RT_diff_no)[:,diff]))
    se_subj_RT_oo.append(np.nanstd(np.array(subj_RT_diff_oo)[:,diff])/np.sqrt(len(sorted_subj_data)))
    Sse_subj_RT_oo.append(np.nanstd(np.array(Ssubj_RT_diff_oo)[:,diff])/np.sqrt(len(sorted_subj_data)))
    se_subj_RT_no.append(np.nanstd(np.array(subj_RT_diff_no)[:,diff])/np.sqrt(len(sorted_subj_data)))
plt.errorbar(np.arange(1,5),mean_subj_RT_oo,se_subj_RT_oo,c='r')
plt.errorbar(np.arange(1,5),mean_subj_RT_no,se_subj_RT_no,c='g')
plt.errorbar(np.arange(1,5),Smean_subj_RT_oo,Sse_subj_RT_oo,c='b')
plt.xlabel('Difficulty')
plt.ylabel('Population mean RT')
plt.legend(("DO optout","non-optout","SO optout"),loc='best', shadow=True)
plt.show()

**Figure 6**: Mean and standard error normalized reaction time of correct answers vs difficulty across participants. Green: non-optout correct trials. Red: DO optout correct trials. Blue: SO optout correct trials.

### Optout vs. difficulty

In [ ]:
ind = -1
fig, ax = plt.subplots(7,4,figsize=(18,25))
plt.subplots_adjust(wspace = 0.3)
plt.subplots_adjust(hspace = 0.3)
subj_optout_oo,Ssubj_optout_oo = [],[]
subj_optout_diff_oo,Ssubj_optout_diff_oo = [],[]
for part in sorted_subj_data:
    optout_diff,Soptout_diff,se_optout_diff,Sse_optout_diff = [[] for _ in range(4)]
    for diff in range(4):
        # optout deterministic
        subset_oob = Ddf_oo[part][(Ddf_oo[part]["difficulty"]==diff)]
        oo_list = list(subset_oob.optout)
        num_trials_oob = len(oo_list)
        optout_diff.append(100*(np.sum(oo_list)/num_trials_oob))
        se_optout_diff.append(100*np.sqrt(optout_diff[diff]/100 *(1-optout_diff[diff]/100)/num_trials_oob))
        # optout stochastic
        Ssubset_oob = Sdf_oo[part][(Sdf_oo[part]["difficulty"]==diff)]
        Soo_list = list(Ssubset_oob.optout)
        Snum_trials_oob = len(Soo_list)
        Soptout_diff.append(100*(np.sum(Soo_list)/Snum_trials_oob))
        Sse_optout_diff.append(100*np.sqrt(Soptout_diff[diff]/100 *(1-Soptout_diff[diff]/100)/Snum_trials_oob))

    subj_optout_diff_oo.append(optout_diff)
    Ssubj_optout_diff_oo.append(Soptout_diff)
    
    subj_optout_oo.append(np.nanmean(optout_diff))
    Ssubj_optout_oo.append(np.nanmean(Soptout_diff))
    
    # write the result in file
    filename=path_fit+'diff_Sub'+str(part)+'_Day'+str(fday)+'_Sess'+str(fsession)+'.json'
    dict_ = {
        "Doptout" : optout_diff,
        "Dse_optout" : se_optout_diff,
        "Soptout" : Soptout_diff,
        "Sse_optout" : Sse_optout_diff    
    }
    # append to the dictionary in the existing file
    # append to the dictionary in the existing file
    with open(filename) as outfile:
        old_data = json.load(outfile)
    old_data.update(dict_)
    with open(filename, 'w') as outfile:
        json.dump(old_data, outfile)
        
    ind += 1
    ind1 = ind%7
    ind2 = int(round(ind/7,1))
    ax[ind1,ind2].set_title('participant:'+str(part))
    ax[ind1,ind2].set_ylim(-5,105)
    ax[ind1,ind2].errorbar(np.arange(1,5),optout_diff,yerr=se_optout_diff,color='r',ls='-')
    ax[ind1,ind2].errorbar(np.arange(1,5),Soptout_diff,yerr=Sse_optout_diff,color='b',ls='-')
    ax[ind1,0].set_ylabel('Optout')
    ax[6,ind2].set_xlabel('Difficulty')
    ax[0,0].legend(("DO optout","SO optout"),loc='best', shadow=True)
    ax[ind1,ind2].set_xticks(np.arange(1,5))
    ax[6,3].axis('off')
    
# write the result in file
dict_ = {
    "Dsubj_optout_oo" : subj_optout_oo,
    "Ssubj_optout_oo" : Ssubj_optout_oo  
}
# append to the dictionary in the existing file
# append to the dictionary in the existing file
with open(filename_average) as outfile:
    old_data = json.load(outfile)
old_data.update(dict_)
with open(filename_average, 'w') as outfile:
    json.dump(old_data, outfile)
plt.show()

**Figure 7**: Mean and standard deviation optout choice vs difficulty for each participant. Red: DO optout trials. Blue: SO optout trials.

In [ ]:
mean_subj_optout_oo,Smean_subj_optout_oo,se_subj_optout_oo,Sse_subj_optout_oo = [[] for _ in range(4)]
for diff in range(4):
    mean_subj_optout_oo.append(np.nanmean(np.array(subj_optout_diff_oo)[:,diff]))
    Smean_subj_optout_oo.append(np.nanmean(np.array(Ssubj_optout_diff_oo)[:,diff]))
    se_subj_optout_oo.append(np.nanstd(np.array(subj_optout_diff_oo)[:,diff])/np.sqrt(len(sorted_subj_data)))
    Sse_subj_optout_oo.append(np.nanstd(np.array(Ssubj_optout_diff_oo)[:,diff])/np.sqrt(len(sorted_subj_data)))
plt.errorbar(np.arange(1,5),mean_subj_optout_oo,se_subj_optout_oo,c='r')
plt.errorbar(np.arange(1,5),Smean_subj_optout_oo,Sse_subj_optout_oo,c='b')
plt.xlabel('Difficulty')
plt.ylabel('Population mean optout')
plt.legend(("DO optout","SO optout"),loc='best', shadow=True)
plt.show()

**Figure 8**: Mean and standard error optout choice vs difficulty across participants. Red: DO optout trials. Blue: SO optout trials.

## Psychometric curves

### NON-optout psychometric curves

Proportion of right answers vs. signed stimuli. 

$y = \frac{1}{1+\exp{-(\beta_0+\beta_1 x)}}$

Sigmoid function as $sigmoid(z)$. 

- Bias: $\beta_0$

- Sensitivity: $ \beta_1 $

If $\beta_0=0$ (no bias), then $y(x=0)=1/2$. If $\beta_0>0$, then then $y(x=0)>1/2$ exposing a bias to rightward answers. Otherwise, there is a bias toward the left option. 

We can estimate the noise in the internal response $\sigma^2$ and the decision boundary $H$ with the non-optout trials. If $x$ is the stimulus strength (i.e. signed signal), the participant observes $\hat{x} = x + \eta$, where $\eta ~ N(0,\sigma^2)$, and the participant answers to the right if $\hat{x}>H$.

Thus, we have $p(rightward|x) = \int _{[H,+ \infty ]} (x+ \eta) d \eta = \int _{[H-x,+\infty]} \eta d \eta = \Phi(\frac{x-H}{\sigma})$ with $\Phi$ as the standard normal cumulative.

By fitting the psychometric curve we could estimate the noise $\sigma$ and the decision boundary $H$. 

$p(rightward|x) = \Phi(\beta_0 + \beta_1 x)$, with $\beta_1=1/\sigma$ and $\beta_0=-H/\sigma$

We fitted the psychometric curve with the logistic regression:

$p(rightward|x) = \frac{1}{1+e^{-(\beta_0+\beta_1 x)}}$, 

with $\sigma=1/\beta_1$ and $H=-\beta_0\sigma$.

In [ ]:
# DO NOT RUN AGAIN

for part in sorted_subj_data:
    # load list with the signed strenght stimuli
    st_no = list(df_no[part]['signed_signal'])
    # stim axis
    unique_stim = df_no[part]['signed_signal'].unique()
    signed_st = sorted(unique_stim)
    # load data
    diff_no = list(df_no[part]['difficulty'])
    orientation_no = list(df_no[part]['orientation'])
    correct_no = list(df_no[part]['discrimination_is_correct'])
    is_right_no = np.array(list(df_no[part]['resp_is_R']))
    # mean and se of righrward answers in non optout trials    
    m_right_No, se_right_No = np.zeros(len(signed_st)),np.zeros(len(signed_st))
    # arrays with answers in non optout trials: 1 rightwars and 0 leftward
    y_no = is_right_no.astype(int)  
    # mean performance (%) over all the stim for each task
    mean_perf_no = np.nanmean(correct_no)*100

    for st in range(len(signed_st)):
        mask_st = (st_no==signed_st[st])
        m_right_No[st] = np.nanmean(y_no[mask_st]==1)
        n_No = np.sum(mask_st)
        se_right_No[st]= np.sqrt(m_right_No[st]*(1-m_right_No[st])/n_No)   

    # reshape of stimuli arrays for the logistic regression
    st_no = np.array(st_no).reshape(-1,1)
    # add a column of ones to the stimuli arrays for the logistic regression
    x_no = [[1,elem[0]] for elem in st_no]
    # lists with beta resulting values of the fit for each type of trials in this 
    # particular session and for this particular participant
    betas_no = []
    # fitted curves
    x_lr = np.arange(-0.2,0.2,0.01)
    
    ### wih logistic regression...
    # y_lr_no = []
    #lr_no = LogisticRegression(C=1000000, fit_intercept=False)
    #lr_no.fit(x_no, y_no)
    #betas_no = lr_no.coef_[0]
    #score_no = np.round(lr_no.score(x_no, y_no),3)
    # sigma: standard deviation of the internal response (with non optout)
    # Hno: decision boundary in the non optout trials
    #Sigma = 1/betas_no[1]
    #Hno = -betas_no[0]*Sigma
    #y_lr_no=myf.sigmoid(betas_no[0]+x_lr*betas_no[1])
    #yRfit = myf.sigmoid(betas_no[0]+np.array(signed_st)*betas_no[1])
    
    # with probit regression...
    pb_no = smf.glm(formula='resp_is_R ~ signed_signal', 
                family=sm.families.Binomial(link = sm.genmod.families.links.probit),
                data=df_no[part]).fit()
    params = pb_no.params
    Hno = -params[0]/params[1]
    Sigma = 1/params[1]
    y_pr_no = [stats.norm.cdf((x-Hno)/Sigma) for x in x_lr]
    yRfit = [stats.norm.cdf((x-Hno)/Sigma) for x in signed_st]
    mse_no=myf.MSE_no(m_right_No,yRfit)
    pearson_chi2=pb_no.pearson_chi2
    
    # write the result in file
    filename=path_fit+'NO_fit_Sub'+str(part)+'_Day'+str(fday)+'_Sess'+str(fsession)+'.json'
    '''
    if excluded_no!=0:
        in_or_out = 'EXCLUIDO'
    else:
        in_or_out = 'OK'
    '''
    dict_ = {
        "Hno":Hno,
        "Sigma":Sigma,
        "signed_st":list(signed_st),
        "m_right_No":list(m_right_No),
        "se_right_No":list(se_right_No),
        "x_lr":list(x_lr),
        "y_pr_no":list(y_pr_no),
        "pearson_chi2":pearson_chi2,
        "mean_perf_no":mean_perf_no,
        "mse_no":mse_no
    }
    # Serializing json  
    json_object = json.dumps(dict_) 

    # Writing to sample.json 
    with open(filename, "w") as outfile: 
        outfile.write(json_object) 


In [ ]:
path_results2 = path_results+'day'+str(fday)+'/session'+str(fsession)+'/'

In [ ]:
# plot

# NO PC fit
NOfit_files = [f for f in os.listdir(path_results2) if f.startswith('NO_fit')]
auxNO = [f.replace('NO_fit_Sub','') for f in NOfit_files]
subj_NOfit = [int(f.replace('_Day'+str(fday)+'_Sess'+str(fsession)+'.json','')) for f in auxNO]
sorted_subj_NOfit = sorted(subj_NOfit)
index_subj_NOfit = [subj_NOfit.index(elem) for elem in sorted_subj_NOfit]
sorted_NOfit_files = [NOfit_files[i] for i in index_subj_NOfit]

ind = -1
fig, ax = plt.subplots(7,4,figsize=(18,25))
plt.subplots_adjust(wspace = 0.3)
plt.subplots_adjust(hspace = 0.4)
for part in sorted_subj_data:
    # psychometric curve NON-optout
    fNO = sorted_NOfit_files[ind]
    filename=path_results2+fNO
    with open(filename) as fNO:
        dataNO = json.load(fNO)
    for k, v in dataNO.items():
        globals()[k]=v     
    ind += 1
    ind1 = ind%7
    ind2 = int(round(ind/7,1))
    ax[ind1,ind2].set_title("Participant "+str(part))
    ax[ind1,ind2].text(0.05, 0.2, 'pearson_chi2='+str(round(pearson_chi2,2)), ha='left', wrap=True)
    ax[ind1,ind2].text(0.05, 0.3, 'bias='+str(round(Hno,2)), ha='left', wrap=True)
    ax[ind1,ind2].errorbar(signed_st,m_right_No,se_right_No,c='g',marker='o',ls='')
    ax[ind1,ind2].plot(x_lr,y_pr_no,c='g')
    ax[0,0].legend(("log-reg","NG NO"),loc='best', shadow=True)
    ax[6,ind2].set_xlabel('signed signal')
    ax[ind1,0].set_ylabel('proportion of right')
    ax[ind1,ind2].set_ylim([-0.1,1.1])
    ax[ind1,ind2].set_xlim([-0.17,0.17])
    ax[ind1,ind2].axvline(0,color='k')
    ax[ind1,ind2].axvline(Hno,color='g',ls='--')
    ax[ind1,ind2].axhline(0.5,color='k')
    ax[ind1,ind2].text(0.05, 0.4,'mean perf:'+str(np.round(mean_perf_no)), ha='left', wrap=True) 
    ax[6,3].axis('off')
    
plt.show()

**Figure 9**: Psychometric curves in non-optout trials for each participant. Dots: mean and sd proportion of rightward choice. Solid line: sigmoid function with logistic regression results. Dashed line: fit resulting decision boundary. Legend meaning of "mean perf": mean performance in non-optout trials, "sigma": fit resulting noise of the internal response, "score": goodness of fit (maximum: 1).

### Deterministic optout psychometric curves

In these trials, the participant has 3 options: L (leftward), R (rightward) and O (opt-out), where the proportion of rightward (leftward) answers vs. the signed stimuli is plotted as psychometric curves. The noise model is assumed to be multinomial, that is:

$p(y_R,y_L,y_O,p_R,p_L,p_O) = \frac{n!}{(y_Ln)!(y_Rn)!(y_On)!}p_L^{y_Ln}p_R^{y_Rn}p_O^{y_On}$, with $y_L+y_R+y_O=1$ and $y_k=\frac{n_k}{n}$ are the fraction of trials where the participant chose each type of response.

Assuming that $p_L \sim 1-\Phi(\frac{x-H_L}{\sigma})$, $p_R \sim \Phi(\frac{x-H_R}{\sigma})$ and $p_O = 1-p_L-p_R$. The stimulus was re-escaled to be bounded within the range $[-1,1]$.

We are going to estimate $H_L,H_R$ and $\sigma$ with maximum likelihood.

### Logistic regression when the participant did not choose the optout

Finally, if the participant did not choose the optout option, then $p_O \sim 0$ and the multinomial probability distribution tend to be binomial. Thus, if the mean optout elections over all the stim is zero, we fit the psychometric curves with the logistic regression over the points where the participant did not choose the optout option.

The x-axis: The 6 presented stimuli ordered as *[l1,l2,l3,r3,r2,r1]*, where 'l' means left, 'r' means right, and the numbers are the corresponding values for difficulty. Right stimuli are positive, while left stimuli are negative. 

In [ ]:
#DO NOT RUN AGAIN
# deterministic optout
  
x_fit=np.linspace(-0.2,0.2,50)
for part in sorted_subj_data:
    # load list with the signed strenght stimuli
    Dst_oo = np.array(list(Ddf_oo[part]['signed_signal']))
    # stim axis
    Dunique_stim = Ddf_oo[part]['signed_signal'].unique()
    Dsigned_st = np.array(sorted(Dunique_stim))
    # load data
    Ddiff_oo = list(Ddf_oo[part]['difficulty'])
    Dorientation_oo = list(Ddf_oo[part]['orientation'])
    Dcorrect_oo = np.array(list(Ddf_oo[part]['discrimination_is_correct']))
    Dis_right_oo = np.array(list(Ddf_oo[part]['resp_is_R']))
    D_oo= list(Ddf_oo[part]['optout'])
    # mean and se of righrward answers in non optout trials    
    Dm_right_oo, Dse_right_oo = np.zeros(len(Dsigned_st)),np.zeros(len(Dsigned_st))
    # arrays with answers in optout trials: 1 rightwars and 0 leftward and 2 for optout
    Dy_oo = np.array(list(Ddf_oo[part]['answer']))
    # mean performance (%) over all the stim for each task
    Dmask_ = (Dy_oo!=2)
    Dmean_perf_oo = np.nanmean(Dcorrect_oo[Dmask_])*100
    Dmean_oo = np.nanmean(D_oo)*100
    # mean and se of answers in optout trials    
    Dm_right_oob,Dse_right_oob,Dm_left_oob,Dse_left_oob,Dm_opt_oob,Dse_opt_oob = [np.zeros(len(Dsigned_st)) for _ in range(6)]
    for st in range(len(Dsigned_st)):
        # this selection sum up 1 with the 3 options: left, right, optout
        Dmask_oob = (Dst_oo==Dsigned_st[st])
        # rightward
        Dm_right_oob[st] = np.nanmean(Dy_oo[Dmask_oob]==1)
        Dn_oob = np.sum(Dmask_oob)
        Dse_right_oob[st]= np.sqrt(Dm_right_oob[st]*(1-Dm_right_oob[st])/Dn_oob)
        # leftward
        Dm_left_oob[st] = np.nanmean(Dy_oo[Dmask_oob]==0)
        Dse_left_oob[st]= np.sqrt(Dm_left_oob[st]*(1-Dm_left_oob[st])/Dn_oob)
        # optout
        Dm_opt_oob[st] = np.nanmean(Dy_oo[Dmask_oob]==2)
        Dse_opt_oob[st]= np.sqrt(Dm_opt_oob[st]*(1-Dm_opt_oob[st])/Dn_oob)
    n=Dst_oo.size

    ## Psychometric fit
    # Exclusion criterion: mean performance higher than 80% and 
    # mean optout election less than 80%
    if Dmean_perf_oo>0 and Dmean_oo<100: 
        # Binomial fit: excluding the optout 
        if Dmean_oo<1:                   
            Dmask_oob2 = (Dy_oo!=2)
            # lists instead of arrays because have variable length in optout trials 
            Dst_oo = Dst_oo[Dmask_oob2]
            # arrays with answers in non optout trials: 1 rightwars and 0 leftward
            Dy_oo=Dy_oo[Dmask_oob2]

            Dresult = myf.fit_oo_binomial('signed_signal','resp_is_R',Ddf_oo[part])
            DHR_oo=Dresult[0][0]
            DHL_oo=Dresult[0][1]
            Dsigma_oo=Dresult[0][2]
            Dexito=Dresult[1]

            # with logistic regression...
            '''
            Dbeta0 = -DHR_oo/Dsigma_oo
            Dbeta1 = 1/Dsigma_oo
            DyR_fit=myf.sigmoid(Dbeta0+x_fit*Dbeta1)
            DyL_fit=1.0-myf.sigmoid(Dbeta0+x_fit*Dbeta1)
            DyR_fit_stim=myf.sigmoid(Dbeta0+Dsigned_st*Dbeta1)
            DyL_fit_stim=1.0-myf.sigmoid(Dbeta0+Dsigned_st*Dbeta1)
            '''

            # with probit regression...
            DyR_fit=[stats.norm.cdf((x-DHR_oo)/Dsigma_oo) for x in x_fit]
            DyL_fit=1.0-np.array([stats.norm.cdf((x-DHR_oo)/Dsigma_oo) for x in x_fit])
            DyO_fit=np.zeros(len(x_fit))   
            # fit result  over the stimuli values
            DyR_fit_stim=[stats.norm.cdf((x-DHR_oo)/Dsigma_oo) for x in Dsigned_st]
            DyL_fit_stim=1.0-np.array([stats.norm.cdf((x-DHR_oo)/Dsigma_oo) for x in Dsigned_st])
            DyO_fit_stim=np.zeros(len(Dsigned_st))
    
            # mean squared error
            Dmse = myf.MSE_oo(Dm_right_oob,Dm_left_oob,Dm_opt_oob,DyR_fit_stim,DyL_fit_stim,DyO_fit_stim)
            DFit = 0
            Dexcluded_oo=0
        # Multinomial fit
        else:
            Dresult = myf.Wrandom_fit_oo_multinomial(Dsigned_st,Dm_right_oob,Dm_left_oob,Dm_opt_oob,n)
            DHR_oo=Dresult[0][0]
            DHL_oo=Dresult[0][1]
            Dsigma_oo=Dresult[0][2]
            Dexito=Dresult[1]
            #print('[H_R  H_L  sigma]=',Dresult[0])
            #print('success:',Dexito)
            DyR_fit=myf.cumul_norm(x_fit,DHR_oo,Dsigma_oo)
            DyL_fit=1.0-myf.cumul_norm(x_fit,DHL_oo,Dsigma_oo)
            DyO_fit=1.0-DyR_fit-DyL_fit  
            # fit result  over the stimuli values
            DyR_fit_stim=myf.cumul_norm(Dsigned_st,DHR_oo,Dsigma_oo)
            DyL_fit_stim=1.0-myf.cumul_norm(Dsigned_st,DHL_oo,Dsigma_oo)
            DyO_fit_stim=1.0-DyR_fit_stim-DyL_fit_stim
            # mean squared error
            Dmse = myf.MSE_oo(Dm_right_oob,Dm_left_oob,Dm_opt_oob,DyR_fit_stim,DyL_fit_stim,DyO_fit_stim)
            DFit = 1
            Dexcluded_oo=0
    elif Dmean_perf_oo<=0:
        DyR_fit=np.zeros(len(x_fit))
        DyL_fit=np.zeros(len(x_fit))
        DyO_fit=np.zeros(len(x_fit))
        # fit result  over the stimuli values
        DyR_fit_stim=np.zeros(len(Dsigned_st))
        DyL_fit_stim=np.zeros(len(Dsigned_st))
        DyO_fit_stim=np.zeros(len(Dsigned_st))
        Dmse = m.nan
        DHR_oo=1000000
        DHL_oo=-1000000
        Dsigma_oo=0
        Dexito = False
        Dexcluded_oo=1
        DFit = 2
    elif Dmean_oo>=100:
        DyR_fit=np.zeros(len(x_fit))
        DyL_fit=np.zeros(len(x_fit))
        DyO_fit=np.ones(len(x_fit))
        DHR_oo=1000000
        DHL_oo=-1000000
        Dsigma_oo=0
        # fit result  over the stimuli values
        DyR_fit_stim=np.zeros(len(Dsigned_st))
        DyL_fit_stim=np.zeros(len(Dsigned_st))
        DyO_fit_stim=np.ones(len(Dsigned_st))
        Dexito = False
        Dmse = m.nan
        DFit = 3
        Dexcluded_oo=1
        
    # write the result in file
    Dfilename=path_fit+'DO_fit_oo_Sub'+str(part)+'_Day'+str(fday)+'_Sess'+str(fsession)+'.json'
    if Dexcluded_oo!=0:
        in_or_out = 'EXCLUIDO'
    else:
        in_or_out = 'OK'
    Ddict_ = {
        "in_or_out":in_or_out,
        "DHR_oo":DHR_oo,
        "DHL_oo":DHL_oo,
        "Dsigma_oo":Dsigma_oo,
        "Dsigned_st":list(Dsigned_st),
        "Dm_right_oob":list(Dm_right_oob),
        "Dm_left_oob":list(Dm_left_oob),
        "Dm_opt_oob":list(Dm_opt_oob),
        "Dse_right_oob":list(Dse_right_oob),
        "Dse_left_oob":list(Dse_left_oob),
        "Dse_opt_oob":list(Dse_opt_oob),
        "x_fit":list(x_fit),
        "DyR_fit":list(DyR_fit),
        "DyL_fit":list(DyL_fit),
        "DyO_fit":list(DyO_fit),
        "DyR_fit_stim":list(DyR_fit_stim),
        "DyL_fit_stim":list(DyL_fit_stim),
        "DyO_fit_stim":list(DyO_fit_stim),
        "n":int(n),
        "Dmse":Dmse,
        "Dexito":Dexito,
        "Dmean_perf_oo":Dmean_perf_oo,
        "Dmean_oo":Dmean_oo,
        "DFit":DFit
    }
    # Serializing json  
    Djson_object = json.dumps(Ddict_) 

    # Writing to sample.json 
    with open(Dfilename, "w") as outfile: 
        outfile.write(Djson_object) 


In [ ]:
Dlist2change = np.load(path_results+'day'+str(fday)+'/session'+str(fsession)+'/DO_sub2changeFit.npy')

In [ ]:
Dlist2change

In [ ]:
# DO not run again 
# this script change the fit from logistic to probit in DO trials where participants did not chose the optout
x_fit=np.linspace(-0.2,0.2,200)

for part in sorted_subj_data:
    # load list with the signed strenght stimuli
    Dst_oo = np.array(list(Ddf_oo[part]['signed_signal']))
    # stim axis
    Dunique_stim = Ddf_oo[part]['signed_signal'].unique()
    Dsigned_st = np.array(sorted(Dunique_stim))
    # load data
    Ddiff_oo = list(Ddf_oo[part]['difficulty'])
    Dorientation_oo = list(Ddf_oo[part]['orientation'])
    Dcorrect_oo = np.array(list(Ddf_oo[part]['discrimination_is_correct']))
    Dis_right_oo = np.array(list(Ddf_oo[part]['resp_is_R']))
    D_oo= list(Ddf_oo[part]['optout'])
    # mean and se of righrward answers in non optout trials    
    Dm_right_oo, Dse_right_oo = np.zeros(len(Dsigned_st)),np.zeros(len(Dsigned_st))
    # arrays with answers in optout trials: 1 rightwars and 0 leftward and 2 for optout
    Dy_oo = np.array(list(Ddf_oo[part]['answer']))
    # mean performance (%) over all the stim for each task
    Dmask_ = (Dy_oo!=2)
    Dmean_perf_oo = np.nanmean(Dcorrect_oo[Dmask_])*100
    Dmean_oo = np.nanmean(D_oo)*100
    # mean and se of answers in optout trials    
    Dm_right_oob,Dse_right_oob,Dm_left_oob,Dse_left_oob,Dm_opt_oob,Dse_opt_oob = [np.zeros(len(Dsigned_st)) for _ in range(6)]
    for st in range(len(Dsigned_st)):
        # this selection sum up 1 with the 3 options: left, right, optout
        Dmask_oob = (Dst_oo==Dsigned_st[st])
        # rightward
        Dm_right_oob[st] = np.nanmean(Dy_oo[Dmask_oob]==1)
        Dn_oob = np.sum(Dmask_oob)
        Dse_right_oob[st]= np.sqrt(Dm_right_oob[st]*(1-Dm_right_oob[st])/Dn_oob)
        # leftward
        Dm_left_oob[st] = np.nanmean(Dy_oo[Dmask_oob]==0)
        Dse_left_oob[st]= np.sqrt(Dm_left_oob[st]*(1-Dm_left_oob[st])/Dn_oob)
        # optout
        Dm_opt_oob[st] = np.nanmean(Dy_oo[Dmask_oob]==2)
        Dse_opt_oob[st]= np.sqrt(Dm_opt_oob[st]*(1-Dm_opt_oob[st])/Dn_oob)
    n=Dst_oo.size

    ## Psychometric fit               
    Dmask_oob2 = (Dy_oo!=2)
    # lists instead of arrays because have variable length in optout trials 
    Dst_oo = Dst_oo[Dmask_oob2]
    # arrays with answers in non optout trials: 1 rightwars and 0 leftward
    Dy_oo=Dy_oo[Dmask_oob2]

    Dresult = myf.fit_oo_binomial('signed_signal','resp_is_R',Ddf_oo[part])
    DHR_oo=Dresult[0][0]
    DHL_oo=Dresult[0][1]
    Dsigma_oo=Dresult[0][2]
    Dexito=Dresult[1]

    # with logistic regression...
    '''
    Dbeta0 = -DHR_oo/Dsigma_oo
    Dbeta1 = 1/Dsigma_oo
    DyR_fit=myf.sigmoid(Dbeta0+x_fit*Dbeta1)
    DyL_fit=1.0-myf.sigmoid(Dbeta0+x_fit*Dbeta1)
    DyR_fit_stim=myf.sigmoid(Dbeta0+Dsigned_st*Dbeta1)
    DyL_fit_stim=1.0-myf.sigmoid(Dbeta0+Dsigned_st*Dbeta1)
    '''

    # with probit regression...
    DyR_fit=[stats.norm.cdf((x-DHR_oo)/Dsigma_oo) for x in x_fit]
    DyL_fit=1.0-np.array([stats.norm.cdf((x-DHR_oo)/Dsigma_oo) for x in x_fit])
    DyO_fit=np.zeros(len(x_fit))   
    # fit result  over the stimuli values
    DyR_fit_stim=[stats.norm.cdf((x-DHR_oo)/Dsigma_oo) for x in Dsigned_st]
    DyL_fit_stim=1.0-np.array([stats.norm.cdf((x-DHR_oo)/Dsigma_oo) for x in Dsigned_st])
    DyO_fit_stim=np.zeros(len(Dsigned_st))
    
    # mean squared error
    Dmse = myf.MSE_oo(Dm_right_oob,Dm_left_oob,Dm_opt_oob,DyR_fit_stim,DyL_fit_stim,DyO_fit_stim)
    DFit = 0
    Dexcluded_oo=0    
    
    filename = 'DO_fit_oo_Sub'+str(part)+'_Day'+str(fday)+'_Sess'+str(fsession)+'.json'
    with open(path_results+'day'+str(fday)+'/session'+str(fsession)+'/'+filename,"r") as f:
        data = json.load(f)
        data["DHR_oo"]=DHR_oo
        data["DHL_oo"]=DHL_oo
        data["Dsigma_oo"]=Dsigma_oo
        data["DyR_fit"]=list(DyR_fit)
        data["DyL_fit"]=list(DyL_fit)
        data["DyR_fit_stim"]=list(DyR_fit_stim)
        data["DyL_fit_stim"]=list(DyL_fit_stim)
        data["Dmse"]=Dmse
        data["Dexito"]=Dexito
     
    with open(path_results+'day'+str(fday)+'/session'+str(fsession)+'/'+filename,"w") as f:
        json.dump(data, f)

In [ ]:
# plot
# Deterministic PC fit
DOfit_files = [f for f in os.listdir(path_results2) if f.startswith('DO_fit')]
aux = [f.replace('DO_fit_oo_Sub','') for f in DOfit_files]
subj_DOfit = [int(f.replace('_Day'+str(fday)+'_Sess'+str(fsession)+'.json','')) for f in aux]
sorted_subj_DOfit = sorted(subj_DOfit)
index_subj_DOfit = [subj_DOfit.index(elem) for elem in sorted_subj_DOfit]
sorted_DOfit_files = [DOfit_files[i] for i in index_subj_DOfit]

ind = -1
fig, ax = plt.subplots(7,4,figsize=(18,25))
plt.subplots_adjust(wspace = 0.3)
plt.subplots_adjust(hspace = 0.4)  
for part in sorted_subj_data:
    ind += 1
    # deterministic PC fit
    f = sorted_DOfit_files[ind]
    filename=path_results2+f
    with open(filename) as f:
        data = json.load(f)
    for k, v in data.items():
        globals()[k]=v        
    ind1 = ind%7
    ind2 = int(round(ind/7,1))
    ax[ind1,ind2].text(-0.14, 1.0, 'mean optout ='+str(np.round(Dmean_oo,2)), ha='left', wrap=True)
    ax[ind1,ind2].text(-0.14, 0.9, 'mean perf ='+str(np.round(Dmean_perf_oo,2)), ha='left', wrap=True)
    ax[ind1,ind2].text(-0.14, 0.8, 'sigma ='+str(np.round(Dsigma_oo,3)), ha='left', wrap=True)
    ax[ind1,ind2].text(-0.14, 0.7, 'mse ='+str(np.round(Dmse,3)), ha='left', wrap=True)
    ax[ind1,ind2].set_title("participant "+str(part))
    ax[ind1,ind2].errorbar(Dsigned_st,Dm_right_oob,Dse_right_oob,c='r',marker='o',ls='')
    ax[ind1,ind2].errorbar(Dsigned_st,Dm_left_oob,Dse_left_oob,c='m',marker='o',ls='')
    ax[ind1,ind2].errorbar(Dsigned_st,Dm_opt_oob,Dse_opt_oob,c='g',marker='o',ls='')
    ax[ind1,ind2].plot(x_fit,DyR_fit,c='r')   
    ax[ind1,ind2].plot(x_fit,DyL_fit,c='m')
    ax[ind1,ind2].plot(x_fit,DyO_fit,c='g')
    ax[0,0].legend(("DO right","DO left","DO opt-out"),loc='best', shadow=True)
    ax[6,ind2].set_xlabel('signed signal')
    ax[ind1,0].set_ylabel('response proportion')
    ax[ind1,ind2].set_ylim([-0.1,1.1])
    ax[ind1,ind2].set_xlim([-0.17,0.17])
    ax[ind1,ind2].axvline(0,color='k')
    ax[ind1,ind2].axvline(DHR_oo,color='r',ls='--')
    ax[ind1,ind2].axvline(DHL_oo,color='m',ls='--')
    ax[ind1,ind2].axhline(0.5,color='k')
    ax[6,3].axis('off')
plt.show()

**Figure 10**: Psychometric curves in DO optout trials for each participant. Red (magenta, green) dots: mean and sd proportion of rightward (leftward, optout) choice. Red (magenta, green) solid line: sigmoid function with maximum likelihood (if mean optout choice > 0) or logistic regression (if mean optout = 0) results for rightward (leftward, optout) choice. Red (magenta, green) dashed line: resulting decision boundary for rightward (leftward, optout) choice. Legend meaning of "mean optout": mean optout election, "mean perf": mean performance in non-optout trials, "sigma": fit resulting noise of the internal response, "mse": mean square error as goodness of fit.

### Stochastic optout psychometric curves

In [ ]:
Slist2change = np.load(path_results+'day'+str(fday)+'/session'+str(fsession)+'/SO_sub2changeFit.npy')

In [ ]:
Slist2change

In [ ]:
x_fit=np.linspace(-0.2,0.2,200)

In [ ]:
# DO not run again 
# this script change the fit from logistic to probit in DO trials where participants did not chose the optout

for part in Slist2change:
    # load list with the signed strenght stimuli
    Sst_oo = np.array(list(Sdf_oo[part]['signed_signal']))
    # stim axis
    Sunique_stim = Sdf_oo[part]['signed_signal'].unique()
    Ssigned_st = np.array(sorted(Sunique_stim))
    # load data
    Sdiff_oo = list(Sdf_oo[part]['difficulty'])
    Sorientation_oo = list(Sdf_oo[part]['orientation'])
    Scorrect_oo = np.array(list(Sdf_oo[part]['discrimination_is_correct']))
    Sis_right_oo = np.array(list(Sdf_oo[part]['resp_is_R']))
    S_oo= list(Sdf_oo[part]['optout'])
    # mean and se of righrward answers in non optout trials    
    Sm_right_oo, Sse_right_oo = np.zeros(len(Ssigned_st)),np.zeros(len(Ssigned_st))
    # arrays with answers in optout trials: 1 rightwars and 0 leftward and 2 for optout
    Sy_oo = np.array(list(Sdf_oo[part]['answer']))
    # mean performance (%) over all the stim for each task
    Smask_ = (Sy_oo!=2)
    Smean_perf_oo = np.nanmean(Scorrect_oo[Smask_])*100
    Smean_oo = np.nanmean(S_oo)*100
    # mean and se of answers in optout trials    
    Sm_right_oob,Sse_right_oob,Sm_left_oob,Sse_left_oob,Sm_opt_oob,Sse_opt_oob = [np.zeros(len(Ssigned_st)) for _ in range(6)]
    for st in range(len(Ssigned_st)):
        # this selection sum up 1 with the 3 options: left, right, optout
        Smask_oob = (Sst_oo==Ssigned_st[st])
        # rightward
        Sm_right_oob[st] = np.nanmean(Sy_oo[Smask_oob]==1)
        Sn_oob = np.sum(Smask_oob)
        Sse_right_oob[st]= np.sqrt(Sm_right_oob[st]*(1-Sm_right_oob[st])/Sn_oob)
        # leftward
        Sm_left_oob[st] = np.nanmean(Sy_oo[Smask_oob]==0)
        Sse_left_oob[st]= np.sqrt(Sm_left_oob[st]*(1-Sm_left_oob[st])/Sn_oob)
        # optout
        Sm_opt_oob[st] = np.nanmean(Sy_oo[Smask_oob]==2)
        Sse_opt_oob[st]= np.sqrt(Sm_opt_oob[st]*(1-Sm_opt_oob[st])/Sn_oob)
    n=np.sum(Sst_oo)

    ## Psychometric fit               
    Smask_oob2 = (Sy_oo!=2)
    # lists instead of arrays because have variable length in optout trials 
    Sst_oo = Sst_oo[Smask_oob2]
    # arrays with answers in non optout trials: 1 rightwars and 0 leftward
    Sy_oo = Sy_oo[Smask_oob2]

    Sresult = myf.fit_oo_binomial('signed_signal','resp_is_R',Sdf_oo[part])
    
    SHR_oo=Sresult[0][0]
    SHL_oo=Sresult[0][1]
    Ssigma_oo=Sresult[0][2]
    Sexito=Sresult[1]

    # with logistic regression...
    '''
    Sbeta0 = -SHR_oo/Ssigma_oo
    Sbeta1 = 1/Ssigma_oo
    SyR_fit=myf.sigmoid(Sbeta0+x_fit*Sbeta1)
    SyL_fit=1.0-myf.sigmoid(Sbeta0+x_fit*Sbeta1)
    SyO_fit=np.zeros(len(x_fit))
    # fit result  over the stimuli values
    SyR_fit_stim=myf.sigmoid(Sbeta0+Ssigned_st*Sbeta1)
    SyL_fit_stim=1.0-myf.sigmoid(Sbeta0+Ssigned_st*Sbeta1)
    SyO_fit_stim=np.zeros(len(Ssigned_st))
    '''

    # with probit regression...
    SyR_fit=[stats.norm.cdf((x-SHR_oo)/Ssigma_oo) for x in x_fit]
    SyL_fit=1.0-np.array([stats.norm.cdf((x-SHR_oo)/Ssigma_oo) for x in x_fit])
    SyO_fit=np.zeros(len(x_fit))   
    # fit result  over the stimuli values
    SyR_fit_stim=[stats.norm.cdf((x-SHR_oo)/Ssigma_oo) for x in Ssigned_st]
    SyL_fit_stim=1.0-np.array([stats.norm.cdf((x-SHR_oo)/Ssigma_oo) for x in Ssigned_st])
    SyO_fit_stim=np.zeros(len(Ssigned_st))
    
    # mean squared error
    Smse = myf.MSE_oo(Sm_right_oob,Sm_left_oob,Sm_opt_oob,SyR_fit_stim,SyL_fit_stim,SyO_fit_stim)  
    
    filename = 'SO_fit_oo_Sub'+str(part)+'_Day'+str(fday)+'_Sess'+str(fsession)+'.json'
    with open(path_results+'day'+str(fday)+'/session'+str(fsession)+'/'+filename,"r") as f:
        data = json.load(f)
        data["SHR_oo"]=SHR_oo
        data["SHL_oo"]=SHL_oo
        data["Ssigma_oo"]=Ssigma_oo
        data["SyR_fit"]=list(SyR_fit)
        data["SyL_fit"]=list(SyL_fit)
        data["SyR_fit_stim"]=list(SyR_fit_stim)
        data["SyL_fit_stim"]=list(SyL_fit_stim)
        data["Smse"]=Smse
        data["Sexito"]=Sexito
     
    with open(path_results+'day'+str(fday)+'/session'+str(fsession)+'/'+filename,"w") as f:
        json.dump(data, f)

In [ ]:
# DO NOT RUN AGAIN!

# stochastic optout
ind = -1 
for part in sorted_subj_data:
    # load list with the signed strenght stimuli
    Sst_oo = np.array(list(Sdf_oo[part]['signed_signal']))
    # stim axis
    Sunique_stim = Sdf_oo[part]['signed_signal'].unique()
    Ssigned_st = np.array(sorted(Sunique_stim))
    # load data
    Sdiff_oo = list(Sdf_oo[part]['difficulty'])
    Sorientation_oo = list(Sdf_oo[part]['orientation'])
    Scorrect_oo = np.array(list(Sdf_oo[part]['discrimination_is_correct']))
    Sis_right_oo = np.array(list(Sdf_oo[part]['resp_is_R']))
    S_oo= list(Sdf_oo[part]['optout'])
    # mean and se of righrward answers in non optout trials    
    Sm_right_oo, Sse_right_oo = np.zeros(len(Ssigned_st)),np.zeros(len(Ssigned_st))
    # arrays with answers in optout trials: 1 rightwars and 0 leftward and 2 for optout
    Sy_oo = np.array(list(Sdf_oo[part]['answer']))
    # mean performance (%) over all the stim for each task
    Smask_ = (Sy_oo!=2)
    Smean_perf_oo = np.nanmean(Scorrect_oo[Smask_])*100
    Smean_oo = np.nanmean(S_oo)*100
    # mean and se of answers in optout trials    
    Sm_right_oob,Sse_right_oob,Sm_left_oob,Sse_left_oob,Sm_opt_oob,Sse_opt_oob = [np.zeros(len(Ssigned_st)) for _ in range(6)]
    for st in range(len(Ssigned_st)):
        # this selection sum up 1 with the 3 options: left, right, optout
        Smask_oob = (Sst_oo==Ssigned_st[st])
        # rightward
        Sm_right_oob[st] = np.nanmean(Sy_oo[Smask_oob]==1)
        Sn_oob = np.sum(Smask_oob)
        Sse_right_oob[st]= np.sqrt(Sm_right_oob[st]*(1-Sm_right_oob[st])/Sn_oob)
        # leftward
        Sm_left_oob[st] = np.nanmean(Sy_oo[Smask_oob]==0)
        Sse_left_oob[st]= np.sqrt(Sm_left_oob[st]*(1-Sm_left_oob[st])/Sn_oob)
        # optout
        Sm_opt_oob[st] = np.nanmean(Sy_oo[Smask_oob]==2)
        Sse_opt_oob[st]= np.sqrt(Sm_opt_oob[st]*(1-Sm_opt_oob[st])/Sn_oob)
    n=np.sum(Sst_oo)

    ## Psychometric fit
    # Exclusion criterion: mean performance higher than 80% and 
    # mean optout election less than 80%
    if Smean_perf_oo>0 and Smean_oo<100: 
        # Binomial fit: excluding the optout 
        if Smean_oo<1:                   
            Smask_oob2 = (Sy_oo!=2)
            # lists instead of arrays because have variable length in optout trials 
            Sst_oo = Sst_oo[Smask_oob2]
            # arrays with answers in non optout trials: 1 rightwars and 0 leftward
            Sy_oo = Sy_oo[Smask_oob2]

            Sresult = myf.fit_oo_binomial('signed_signal','resp_is_R',Sdf_oo[part])

            SHR_oo=Sresult[0][0]
            SHL_oo=Sresult[0][1]
            Ssigma_oo=Sresult[0][2]
            Sexito=Sresult[1]

            # with logistic regression...
            '''
            Sbeta0 = -SHR_oo/Ssigma_oo
            Sbeta1 = 1/Ssigma_oo
            SyR_fit=myf.sigmoid(Sbeta0+x_fit*Sbeta1)
            SyL_fit=1.0-myf.sigmoid(Sbeta0+x_fit*Sbeta1)
            SyO_fit=np.zeros(len(x_fit))
            # fit result  over the stimuli values
            SyR_fit_stim=myf.sigmoid(Sbeta0+Ssigned_st*Sbeta1)
            SyL_fit_stim=1.0-myf.sigmoid(Sbeta0+Ssigned_st*Sbeta1)
            SyO_fit_stim=np.zeros(len(Ssigned_st))
            '''

            # with probit regression...
            SyR_fit=[stats.norm.cdf((x-SHR_oo)/Ssigma_oo) for x in x_fit]
            SyL_fit=1.0-np.array([stats.norm.cdf((x-SHR_oo)/Ssigma_oo) for x in x_fit])
            SyO_fit=np.zeros(len(x_fit))   
            # fit result  over the stimuli values
            SyR_fit_stim=[stats.norm.cdf((x-SHR_oo)/Ssigma_oo) for x in Ssigned_st]
            SyL_fit_stim=1.0-np.array([stats.norm.cdf((x-SHR_oo)/Ssigma_oo) for x in Ssigned_st])
            SyO_fit_stim=np.zeros(len(Ssigned_st))
    
            # mean squared error
            Smse = myf.MSE_oo(Sm_right_oob,Sm_left_oob,Sm_opt_oob,SyR_fit_stim,SyL_fit_stim,SyO_fit_stim)
            SFit = 0
            Sexcluded_oo=0
        # Multinomial fit
        else:
            Sresult = myf.Wrandom_fit_oo_multinomial(Ssigned_st,Sm_right_oob,Sm_left_oob,Sm_opt_oob,n)
            SHR_oo=Sresult[0][0]
            SHL_oo=Sresult[0][1]
            Ssigma_oo=Sresult[0][2]
            Sexito=Sresult[1]
            #print('[H_R  H_L  sigma]=',Sresult[0])
            #print('success:',Sexito)
            SyR_fit=myf.cumul_norm(x_fit,SHR_oo,Ssigma_oo)
            SyL_fit=1.0-myf.cumul_norm(x_fit,SHL_oo,Ssigma_oo)
            SyO_fit=1.0-SyR_fit-SyL_fit  
            # fit result  over the stimuli values
            SyR_fit_stim=myf.cumul_norm(Ssigned_st,SHR_oo,Ssigma_oo)
            SyL_fit_stim=1.0-myf.cumul_norm(Ssigned_st,SHL_oo,Ssigma_oo)
            SyO_fit_stim=1.0-SyR_fit_stim-SyL_fit_stim
            # mean squared error
            Smse = myf.MSE_oo(Sm_right_oob,Sm_left_oob,Sm_opt_oob,SyR_fit_stim,SyL_fit_stim,SyO_fit_stim)
            SFit = 1
            Sexcluded_oo=0
    elif Smean_perf_oo<=0:
        SyR_fit=np.zeros(len(x_fit))
        SyL_fit=np.zeros(len(x_fit))
        SyO_fit=np.zeros(len(x_fit))
        # fit result  over the stimuli values
        SyR_fit_stim=np.zeros(len(Ssigned_st))
        SyL_fit_stim=np.zeros(len(Ssigned_st))
        SyO_fit_stim=np.zeros(len(Ssigned_st))
        Smse = m.nan
        SHR_oo=1000000
        SHL_oo=-1000000
        Ssigma_oo=0
        Sexito = False
        Sexcluded_oo=1
        SFit = 2
    elif Smean_oo>=100:
        SyR_fit=np.zeros(len(x_fit))
        SyL_fit=np.zeros(len(x_fit))
        SyO_fit=np.ones(len(x_fit))
        SHR_oo=1000000
        SHL_oo=-1000000
        Sexito = False
        Ssigma_oo=0
        # fit result  over the stimuli values
        SyR_fit_stim=np.zeros(len(Ssigned_st))
        SyL_fit_stim=np.zeros(len(Ssigned_st))
        SyO_fit_stim=np.ones(len(Ssigned_st))
        Smse = m.nan
        SFit = 3
        Sexcluded_oo=1

    # write the result in file
    Sfilename=path_fit+'SO_fit_oo_Sub'+str(part)+'_Day'+str(fday)+'_Sess'+str(fsession)+'.json'
    if Sexcluded_oo!=0:
        Sin_or_out = 'EXCLUIDO'
    else:
        Sin_or_out = 'OK'
    Sdict_ = {
        "Sin_or_out":Sin_or_out,
        "SHR_oo":SHR_oo,
        "SHL_oo":SHL_oo,
        "Ssigma_oo":Ssigma_oo,
        "Ssigned_st":list(Ssigned_st),
        "Sm_right_oob":list(Sm_right_oob),
        "Sm_left_oob":list(Sm_left_oob),
        "Sm_opt_oob":list(Sm_opt_oob),
        "Sse_right_oob":list(Sse_right_oob),
        "Sse_left_oob":list(Sse_left_oob),
        "Sse_opt_oob":list(Sse_opt_oob),
        "x_fit":list(x_fit),
        "SyR_fit":list(SyR_fit),
        "SyL_fit":list(SyL_fit),
        "SyO_fit":list(SyO_fit),
        "SyR_fit_stim":list(SyR_fit_stim),
        "SyL_fit_stim":list(SyL_fit_stim),
        "SyO_fit_stim":list(SyO_fit_stim),
        "n":int(n),
        "Smse":Smse,
        "Sexito":Sexito,
        "Smean_perf_oo":Smean_perf_oo,
        "Smean_oo":Smean_oo,
        "SFit":SFit
    }
    # Serializing json  
    Sjson_object = json.dumps(Sdict_) 

    # Writing to sample.json 
    with open(Sfilename, "w") as outfile: 
        outfile.write(Sjson_object) 
        

In [ ]:
# plots
# Stochastic PC fit
SOfit_files = [f for f in os.listdir(path_results2) if f.startswith('SO_fit')]
auxSO = [f.replace('SO_fit_oo_Sub','') for f in SOfit_files]
subj_SOfit = [int(f.replace('_Day'+str(fday)+'_Sess'+str(fsession)+'.json','')) for f in auxSO]
sorted_subj_SOfit = sorted(subj_SOfit)
index_subj_SOfit = [subj_SOfit.index(elem) for elem in sorted_subj_SOfit]
sorted_SOfit_files = [SOfit_files[i] for i in index_subj_SOfit]

fig, ax = plt.subplots(7,4,figsize=(18,25))
plt.subplots_adjust(wspace = 0.3)
plt.subplots_adjust(hspace = 0.4)  
ind = -1 
for part in sorted_subj_data:
    ind += 1 
    # stochastic PC fit
    fSO = sorted_SOfit_files[ind]
    filename=path_results2+fSO
    with open(filename) as fSO:
        dataSO = json.load(fSO)
    for k, v in dataSO.items():
        globals()[k]=v  
    if part==3076:
        print(len(x_fit))
    ind1 = ind%7
    ind2 = int(round(ind/7,1))
    ax[ind1,ind2].text(-0.14, 1.0, 'mean optout ='+str(np.round(Smean_oo,2)), ha='left', wrap=True)
    ax[ind1,ind2].text(-0.14, 0.9, 'mean perf ='+str(np.round(Smean_perf_oo,2)), ha='left', wrap=True)
    ax[ind1,ind2].text(-0.14, 0.8, 'sigma ='+str(np.round(Ssigma_oo,3)), ha='left', wrap=True)
    ax[ind1,ind2].text(-0.14, 0.7, 'mse ='+str(np.round(Smse,3)), ha='left', wrap=True)
    ax[ind1,ind2].set_title("participant "+str(part))
    ax[ind1,ind2].errorbar(Ssigned_st,Sm_right_oob,Sse_right_oob,c='b',marker='o',ls='')
    ax[ind1,ind2].errorbar(Ssigned_st,Sm_left_oob,Sse_left_oob,c='c',marker='o',ls='')
    ax[ind1,ind2].errorbar(Ssigned_st,Sm_opt_oob,Sse_opt_oob,c='y',marker='o',ls='')
    ax[ind1,ind2].plot(x_fit,SyR_fit,c='b')   
    ax[ind1,ind2].plot(x_fit,SyL_fit,c='c')
    ax[ind1,ind2].plot(x_fit,SyO_fit,c='y')
    ax[0,0].legend(("SO right","SO left","SO opt-out"),loc='best', shadow=True)
    ax[6,ind2].set_xlabel('signed signal')
    ax[ind1,0].set_ylabel('response proportion')
    ax[ind1,ind2].set_ylim([-0.1,1.1])
    ax[ind1,ind2].set_xlim([-0.17,0.17])
    ax[ind1,ind2].axvline(0,color='k')
    ax[ind1,ind2].axvline(SHR_oo,color='b',ls='--')
    ax[ind1,ind2].axvline(SHL_oo,color='c',ls='--')
    ax[ind1,ind2].axhline(0.5,color='k')
    ax[6,3].axis('off')
plt.show()

**Figure 11**: Psychometric curves in SO optout trials for each participant. Blue (cyan, yellow) dots: mean and sd proportion of rightward (leftward, optout) choice. Blue (cyan, yellow) solid line: sigmoid function with maximum likelihood (if mean optout choice > 0) or logistic regression (if mean optout = 0) results for rightward (leftward, optout) choice. Blue (cyan, yellow) dashed line: resulting decision boundary for rightward (leftward, optout) choice. Legend meaning of "mean optout": mean optout election, "mean perf": mean performance in non-optout trials, "sigma": fit resulting noise of the internal response, "mse": mean square error as goodness of fit.

## Winners

Since the first and second winner of each session receive extra payment, we compute the score. 

In [ ]:
total_score = []
print('part','score')
for part in sorted_subj_data:
    total_score.append(np.sum(Ddf[part]['score'])+np.sum(Sdf[part]['score']))
    print(part,np.sum(Ddf[part]['score'])+np.sum(Sdf[part]['score']))

**Table 2**: Score table. Column 'part' with participant number ID. Column 'score' with the total amount of points for each participant. 

In [ ]:
winners = np.zeros(4)
winners[0] = fday
winners[1] = fsession

In [ ]:
sorted_total_score = sorted(total_score, reverse=True)
if len(np.where(total_score==sorted_total_score[0])[0])==2:
    winners[2] = sorted_subj_data[np.where(total_score==sorted_total_score[0])[0][0]]
    winners[3] = sorted_subj_data[np.where(total_score==sorted_total_score[0])[0][1]]
elif len(np.where(total_score==sorted_total_score[0])[0])>2:
    print('más de dos ganadores!!!')
else:
    winners[2] = sorted_subj_data[total_score.index(sorted_total_score[0])]
    winners[3] = sorted_subj_data[total_score.index(sorted_total_score[1])]

In [ ]:
print('winners of the current session:',int(winners[2]),int(winners[3]))